In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import preprocessing


### handle 資料狀況


In [7]:

# Load the CSV file into a DataFrame
df = pd.read_csv('ds_clean/specific_ds_clean.csv')
df

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01 06:31:08,0.0,1016.50,17.50,86.80,25.00,0.00
1,1,2024-01-01 06:32:08,0.0,1016.53,17.50,86.70,28.33,0.00
2,1,2024-01-01 06:33:08,0.0,1016.57,17.50,86.70,32.50,0.00
3,1,2024-01-01 06:34:08,0.0,1016.58,17.50,86.70,39.17,0.00
4,1,2024-01-01 06:35:08,0.0,1016.59,17.50,86.70,45.83,0.00
...,...,...,...,...,...,...,...,...
226848,3,2024-07-10 11:06:07,0.0,1003.94,47.05,38.20,117758.20,1489.19
226849,3,2024-07-10 11:07:07,0.0,1003.96,47.07,38.27,117758.20,1491.42
226850,3,2024-07-10 11:08:07,0.0,1003.99,47.15,37.53,117758.20,1496.63
226851,3,2024-07-10 11:09:07,0.0,1003.90,46.53,38.03,117758.20,1506.62


#### 把資料的時間改成從某點開始算的方式

In [8]:
# Convert the 'DateTime' column to datetime format
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Extract the date part and calculate the day of the year
df['DayOfYear'] = df['DateTime'].dt.dayofyear
# Calculate the number of minutes since the start of the day
df['MinutesSinceStartOfDay'] = df['DateTime'].dt.hour * 60 + df['DateTime'].dt.minute
# Display the updated DataFrame
df = df.drop(columns=['DateTime'])
df

,LocationCode,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),DayOfYear,MinutesSinceStartOfDay
0,1,0.0,1016.50,17.50,86.80,25.00,0.00,1,391
1,1,0.0,1016.53,17.50,86.70,28.33,0.00,1,392
2,1,0.0,1016.57,17.50,86.70,32.50,0.00,1,393
3,1,0.0,1016.58,17.50,86.70,39.17,0.00,1,394
4,1,0.0,1016.59,17.50,86.70,45.83,0.00,1,395
...,...,...,...,...,...,...,...,...,...
226848,3,0.0,1003.94,47.05,38.20,117758.20,1489.19,192,666
226849,3,0.0,1003.96,47.07,38.27,117758.20,1491.42,192,667
226850,3,0.0,1003.99,47.15,37.53,117758.20,1496.63,192,668
226851,3,0.0,1003.90,46.53,38.03,117758.20,1506.62,192,669


#### 策略
因為目前看起來是要找同時間的周遭測站的狀況去推估發電情形(不然就變成是猜天氣)
所以應該會需要先過濾出能train的資料